In [ ]:
import numpy as np #computação numérica como vetores
import pandas as pd #análise e processamento de dados
import matplotlib.pylab as plt #visualização de dados
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 10,6

In [ ]:
dataset = pd.read_csv("AirPassengers.csv")
#dataset.head
#dataset.get('Month',default="erro")

#converte string para o tipo datetime
dataset['Month'] = pd.to_datetime(dataset['Month'], infer_datetime_format=True)
indexedDataset = dataset.set_index(['Month'])

In [ ]:
from datetime import datetime
indexedDataset.head(5)

In [ ]:
indexedDataset.tail(5)

In [ ]:
##plotar gráfico

plt.xlabel("Data")
plt.ylabel("Número de passageiros")
plt.plot(indexedDataset)

In [ ]:
#determinando as estatísticas móveis
#média
rolmean = indexedDataset.rolling(window=12).mean()

#desvio padrão
rolstd = indexedDataset.rolling(window=12).std()
print(rolmean, rolstd)

#como estamos analisando dados mês a mês, usamos a janela 12
#caso fôssemos analisar dia a dia, a janela poderia ser 365

In [ ]:
#plotando as estatísticas móveis
orig = plt.plot(indexedDataset, color='blue',label='Original')
mean = plt.plot(rolmean, color='red',label='Média Móvel')
std = plt.plot(rolstd, color='black',label='Desvio Padrão Móvel')
plt.legend(loc='best')
plt.title('Média Móvel e Desvio Padrão')
plt.show(block=False)

In [ ]:
#teste de Dickey-Fuller
from statsmodels.tsa.stattools import adfuller

print('Resultados do Teste Dickey-Fuller: ')
dftest = adfuller(indexedDataset['Passengers'], autolag='AIC')

#pense no AIC descrito acima como uma métrica

dfoutput = pd.Series(dftest[0:4], index=['Estatística de teste','p-valor','#lags usadas','número de observações usadas'])
for key,value in dftest[4].items():
    dfoutput['Valor Crítico (%s)'%key] = value

print(dfoutput)

#um p-valor deve normalmente estar em torno de 0.5
#valor crítico deve ser menor que a estatística de teste

In [ ]:
#estimando a tendência
indexedDataset_logScale = np.log(indexedDataset)
plt.plot(indexedDataset_logScale)

In [ ]:
movingAverage = indexedDataset_logScale.rolling(window=12).mean()
movingSTD = indexedDataset_logScale.rolling(window=12).std()
plt.plot(indexedDataset_logScale)
plt.plot(movingAverage, color='red')

In [ ]:
datasetLogScaleMinusMovingAverage = indexedDataset_logScale - movingAverage
datasetLogScaleMinusMovingAverage.head(12)

#remover valores NaN
datasetLogScaleMinusMovingAverage.dropna(inplace=True)
datasetLogScaleMinusMovingAverage.head(10)

In [ ]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):

    #determinar as estatísticas móveis
    movingAverage = timeseries.rolling(window=12).mean()
    movingSTD = timeseries.rolling(window=12).std()
    
    #plotar as estatísticas móveis
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(movingAverage, color='red',label='Média Móvel')
    std = plt.plot(movingSTD, color='black',label='Desvio Padrão Móvel')
    plt.legend(loc='best')
    plt.title('Média Móvel e Desvio Padrão')
    plt.show(block=False)
    
    #teste de Dickey-Fuller
    print('Resultados do Teste Dickey-Fuller: ')
    dftest = adfuller(timeseries['Passengers'], autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Estatística de teste','p-valor','#lags usadas','número de observações usadas'])
    for key,value in dftest[4].items():
        dfoutput['Valor Crítico (%s)'%key] = value
    print(dfoutput)

In [ ]:
test_stationarity(datasetLogScaleMinusMovingAverage)

In [ ]:
exponentialDecayWeightedAverage = indexedDataset_logScale.ewm(halflife=12, min_periods=0, adjust=True).mean()
plt.plot(indexedDataset_logScale)
plt.plot(exponentialDecayWeightedAverage, color='red')

In [ ]:
datasetLogScaleMinusMovingExponentialDecayAverage = indexedDataset_logScale - exponentialDecayWeightedAverage
test_stationarity(datasetLogScaleMinusMovingExponentialDecayAverage)

In [ ]:
datasetLogDiffShifting = indexedDataset_logScale - indexedDataset_logScale.shift()
plt.plot(datasetLogDiffShifting)

In [ ]:
datasetLogDiffShifting.dropna(inplace=True)
test_stationarity(datasetLogDiffShifting)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(indexedDataset_logScale)

#tendência
trend = decomposition.trend
#sazonal
seasonal = decomposition.seasonal
#residual
residual = decomposition.resid

plt.subplot(411)
plt.plot(indexedDataset_logScale,label='Original')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend,label='Tendência')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal,label='Sazonalidade')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual,label='Residual')
plt.legend(loc='best')
plt.tight_layout()

In [ ]:
#### ERRO ABAIXO - PENDENTE DE SOLUÇÃO
decomposedLogData = residual
decomposedLogData.dropna(inplace=True)
test_stationarity(decomposedLogData)

In [ ]:
#acf and pacf plots
from statsmodels.tsa.stattools import acf, pacf

lag_acf = acf(datasetLogDiffShifting, nlags=20)
lag_pacf = pacf(datasetLogDiffShifting, nlags=20, method='ols')

#plotagem ACF
plt.subplot(121)
plt.plot(lag_acf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(datasetLogDiffShifting)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(datasetLogDiffShifting)),linestyle='--',color='gray')
plt.title('Função de autocorrelação')

#plotagem PACF
plt.subplot(122)
plt.plot(lag_pacf)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(datasetLogDiffShifting)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(datasetLogDiffShifting)),linestyle='--',color='gray')
plt.title('Função de autocorrelação parcial')
plt.tight_layout()

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

#AR model
model = ARIMA(indexedDataset_logScale, order=(2, 1, 2))
results_AR = model.fit(disp=-1)
plt.plot(datasetLogDiffShifting)
plt.plot(results_AR.fittedvalues, color='red')
plt.title('RSS: %.4f'% sum((results_AR.fittedvalues-datasetLogDiffShifting["Passengers"])**2))
print('Plotando AR model')

# RSS = soma residual dos quadrados
# quanto maior o RSS, pior

In [ ]:
#MA model
model = ARIMA(indexedDataset_logScale, order=(0, 1, 2))
results_MA = model.fit(disp=-1)
plt.plot(datasetLogDiffShifting)
plt.plot(results_MA.fittedvalues, color='red')
plt.title('RSS: %.4f'% sum((results_MA.fittedvalues-datasetLogDiffShifting["Passengers"])**2))
print('Plotando MA model')

In [ ]:
#ARIMA model
model = ARIMA(indexedDataset_logScale, order=(2, 1, 2))
results_ARIMA = model.fit(disp=-1)
plt.plot(datasetLogDiffShifting)
plt.plot(results_ARIMA.fittedvalues, color='red')
plt.title('RSS: %.4f'% sum((results_ARIMA.fittedvalues-datasetLogDiffShifting["Passengers"])**2))

In [ ]:
predictions_ARIMA_diff = pd.Series(results_ARIMA.fittedvalues, copy=True)
print(predictions_ARIMA_diff.head())

In [ ]:
#converter para soma acumulative
predictions_ARIMA_diff_cumsum = predictions_ARIMA_diff.cumsum()
print(predictions_ARIMA_diff_cumsum.head())

In [ ]:
predictions_ARIMA_log = pd.Series(indexedDataset_logScale['Passengers'].iloc[0], index=indexedDataset_logScale.index)
predictions_ARIMA_log = predictions_ARIMA_log.add(predictions_ARIMA_diff_cumsum,fill_value=0)
predictions_ARIMA_log.head()

In [ ]:
predictions_ARIMA = np.exp(predictions_ARIMA_log)
plt.plot(indexedDataset)
plt.plot(predictions_ARIMA)

In [ ]:
indexedDataset_logScale

In [ ]:
results_ARIMA.plot_predict(1,264)
x = results_ARIMA.forecast(steps=120)

In [ ]:
x[1]

In [ ]:
len(x[1])

In [ ]:
np.exp(x[1])